# Articles
https://keras.io/guides/transfer_learning/
https://keras.io/guides/training_with_built_in_methods/
https://www.youtube.com/watch?v=4umFSRPx-94&ab_channel=DigitalSreeni
https://www.tensorflow.org/guide/saved_model
https://github.com/UCSD-E4E/PyHa/blob/Microfaune_Retraining/Microfaune_Retraining-Copy1.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, Flatten, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import numpy as np


%matplotlib notebook
#add comments
import os
import sys
import csv
import pickle
from collections import Counter
from datetime import datetime

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit

import torch
from torch import nn
from torch.utils.data import DataLoader

import librosa
from librosa import display
from microfaune.audio import wav2spc, create_spec, load_wav
from glob import glob

from torch.utils.data import Dataset


from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import scipy
import IPython.display as ipd

#from TweetyNetModel import TweetyNetModel  


In [3]:
train = True

## Preprocess and load data
According to `config.py` and `checkpoints/README.md` for V2.1:  
- Model training input size: 144000 = 48000 x 3 = sample rate x num chunks  
- Model training output size: 2434 = 2424 bird classes + 10 non-event classes
- Visualize using [Netron](https://netron.app/)

In [4]:
train = True
fineTuning = False
#needs at least 80 for mel spectrograms ## may be able to do a little less, but must be greater than 60
n_mels=72 # The closest we can get tmeporally is 72 with an output of 432 : i think it depends on whats good
#this number should be proportional to the length of the videos. 
datasets_dir = "../passive-acoustic-biodiversity/TweetyNET/data"

In [5]:
def show_mel_spectrogram(spec, uid):
    fig, ax = plt.subplots()
    M_db_bird = librosa.power_to_db(spec, ref=np.max)
    img = librosa.display.specshow(M_db_bird, y_axis='mel', x_axis='time', ax=ax)
    ax.set(title=uid)
    plt.show()

In [8]:
import librosa

def splitSignal(sig, rate, overlap, seconds=3.0, minlen=1.5):

    # Split signal with overlap
    sig_splits = []
    for i in range(0, len(sig), int((seconds - overlap) * rate)):
        split = sig[i:i + int(seconds * rate)]

        # End of signal?
        if len(split) < int(minlen * rate):
            break
        
        # Signal chunk too short? Fill with zeros.
        if len(split) < int(rate * seconds):
            temp = np.zeros((int(rate * seconds)))
            temp[:len(split)] = split
            split = temp
        
        sig_splits.append(split)

    return sig_splits

def readAudioData(path, overlap, sample_rate=48000):

    #print('READING AUDIO DATA...', end=' ', flush=True)

    # Open file with librosa (uses ffmpeg or libav)
    try:
        sig, rate = librosa.load(path, sr=sample_rate, mono=True, res_type='kaiser_fast')
        clip_length = librosa.get_duration(y=sig, sr=rate)
    except:
        return 0
    # Split audio into 3-second chunks
    chunks = splitSignal(sig, rate, overlap)

    #rint('DONE! READ', str(len(chunks)), 'CHUNKS.')

    return chunks, clip_length

def load_dataset(data_path, use_dump=True):
    mel_dump_file = os.path.join(data_path, "mel_dataset.pkl")
    if os.path.exists(mel_dump_file) and use_dump:
        with open(mel_dump_file, "rb") as f:
            dataset = pickle.load(f)
    else:
        dataset = compute_feature(data_path)
        with open(mel_dump_file, "wb") as f:
            pickle.dump(dataset, f)
    inds = [i for i, x in enumerate(dataset["X"])]
    Y = np.array([int(dataset["Y"][i]) for i in inds])
    uids = [dataset["uids"][i] for i in inds]
    X = np.array(dataset["X"]).transpose()
    del dataset
    return X, Y, uids

def compute_feature(data_path):
    print(f"Compute features for dataset {os.path.basename(data_path)}")
    labels_file = os.path.join(data_path, "labels.csv")
    print(labels_file)
    if os.path.exists(labels_file):
        with open(labels_file, "r") as f:
            reader = csv.reader(f, delimiter=',')
            labels = {}
            next(reader)  # pass fields names
            for name, _, y in reader:
                labels[name] = y
    else:
        print("Warning: no label file detected.")
        wav_files = glob(os.path.join(data_path, "wav/*.wav"))
        labels = {os.path.basename(f)[:-4]: None for f in wav_files}
    i = 1
    X = []
    Y = []
    uids = []
    for file_id, y in labels.items():
        print(y)
        print(f"{i:04d}/{len(labels)}: {file_id:20s}", end="\r")
        spc = wav2spc(os.path.join(data_path, "wav", f"{file_id}.wav"), n_mels=n_mels)
        chunks, clip_length = readAudioData(os.path.join(data_path, "wav", f"{file_id}.wav"), 0, sample_rate=48000)
        for c in range(len(chunks)):
            X.append(chunks[c])
            Y.append(y)
            uids.append(file_id)
           
    return {"uids": uids, "X": X, "Y": Y}

def split_dataset(X, Y, test_size=0.2, random_state=0):
    split_generator = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    ind_train, ind_test = next(split_generator.split(X, Y))
    X_train, X_test = X[ind_train, :, :], X[ind_test, :, :]
    Y_train, Y_test = Y[ind_train], Y[ind_test]
    return ind_train, ind_test

In [9]:
datasets_dir = "./Data"
X0, Y0, uids0 = load_dataset(os.path.join(datasets_dir, "ff1010bird_wav"))
#X1, Y1, uids1 = load_dataset(os.path.join(datasets_dir, "warblrb10k_public_wav"))
print(X0.shape, X1.shape)
print(Y0.shape, Y1.shape)
print(len(uids0), len(uids1))

X = np.concatenate([X0, X1]).astype(np.float32)/255
Y = np.concatenate([Y0, Y1])
uids = np.concatenate([uids0, uids1])
print(X.shape, Y.shape, uids.shape)
del X0, Y0, uids0, X1, Y1, uids1 

MemoryError: Unable to allocate 25.8 GiB for an array with shape (24046, 144000) and data type float64

In [ ]:
Counter(Y)

In [ ]:
ind_train_val, ind_test = split_dataset(X, Y)
ind_train, ind_val = split_dataset(X[ind_train_val, :, :, np.newaxis], Y[ind_train_val], test_size=0.1)
X_train, X_test, X_val = X[ind_train, :, :, np.newaxis], X[ind_test, :, :, np.newaxis], X[ind_val, :, :, np.newaxis]
Y_train, Y_test, Y_val = Y[ind_train], Y[ind_test], Y[ind_val]
uids_train, uids_test, uids_val = uids[ind_train], uids[ind_test], uids[ind_val]
del X, Y

print("Training set: ", Counter(Y_train))
print("Test set: ", Counter(Y_test))
print("Validation set: ", Counter(Y_val))
#very imbalanced dataset

## Load in BirdNET-Analyzer Model

In [ ]:
# path to model folder, should have assets/variables/frozen pb graphs
path_to_saved_model = "./checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_Model"
model = tf.keras.models.load_model(path_to_saved_model)

In [ ]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=['recall'])

In [ ]:
x = model.layers[-2].output
#o = tf.keras.layers.Dense(len(species_of_interests), activation='sigmoid', name='its_new_lmao')(x)
o = tf.keras.layers.Dense(2, activation='sigmoid', name='its_new_lmao')(x)

model3 = Model(inputs=model.input, outputs=o)
#model.get_weights()

In [ ]:
model3.get_weights()

In [ ]:
len(species_of_interests)

In [ ]:
model_extended = model3
model_extended

In [ ]:
model_extended.summary()

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)


## Retrain Model

In [ ]:
from datetime import datetime
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, Y, batch_size=32, output_size=10):
        self.X = X
        self.Y = Y
        self.n = len(Y)
        self.batch_size = batch_size
        self.output_size = output_size
        self.shuffle()
        
        
    def __len__(self):
        print("RUNNINGS", int(np.floor(self.n)/self.batch_size))
        return int(np.floor(self.n)/self.batch_size)
    
    def __getitem__(self, index):
        
        batch_inds = self.inds[self.batch_size*index:self.batch_size*(index+1)]
        tensor = tf.convert_to_tensor(self.X[batch_inds, ...][0])
        #print(index, tf.reduce_max(tensor), tf.reduce_min(tensor), tf.reduce_max(tensor) == tf.reduce_min(tensor))
        if (tf.reduce_max(tensor) == tf.reduce_min(tensor)):
            try:
                print("REDO")
                print(index, tf.reduce_max(tensor), tf.reduce_min(tensor), tf.reduce_max(tensor) == tf.reduce_min(tensor))
                batch_inds = self.inds[self.batch_size*(index+1):self.batch_size*(index+ 2)]
                tensor = tf.convert_to_tensor(self.X[batch_inds, ...][0])
                print(index+1, tf.reduce_max(tensor), tf.reduce_min(tensor), tf.reduce_max(tensor) == tf.reduce_min(tensor))
            except:
                batch_inds = self.inds[self.batch_size*(index-1):self.batch_size*(index)]
                tensor = tf.convert_to_tensor(self.X[batch_inds, ...][0])

        tensor = tf.math.divide(
           tf.math.subtract(
              tensor, 
              tf.reduce_min(tensor)
           ), 
           tf.math.subtract(
              tf.reduce_max(tensor), 
              tf.reduce_min(tensor)
           )
        )
        
        
        
        #print(batch_inds)
        self.counter += self.batch_size
        if self.counter >= self.n:
            self.shuffle()
            
        #TODO: FIX SO BATCH IS MORE THAN 1
        raw_labels = np.array([self.Y[batch_inds][0]])
        formatted_labels = np.zeros(self.output_size)
        formatted_labels[self.Y[batch_inds][0]] = 1
        fprmatted_labels = np.array([formatted_labels])
        #print(formatted_labels.shape)
        #print(np.array([formatted_labels]).shape)
        #print(self.X[batch_inds, ...][0].shape)

        
        #print(tf.convert_to_tensor(self.X[batch_inds, ...][0]), tf.convert_to_tensor(np.array([formatted_labels])))
        return tensor, tf.convert_to_tensor(np.array([formatted_labels]))
    
    def shuffle(self):
        self.inds = np.random.permutation(self.n)
        self.counter = 0
# if train:

In [ ]:
len(species_of_interests)

In [ ]:
X_train[0].shape

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, Flatten, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import numpy as np

# microfaune training script

if train:
    optimizer = keras.optimizers.Adam(lr=0.001)
    model_extended.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy', keras.metrics.FalseNegatives()])

    alpha = 0.5
    batch_size = 1
    date_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    data_generator = DataGenerator(X_train, Y_train, batch_size, output_size=2)
    
    micro_callbacks = [
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                  patience=5, min_lr=1e-5),
        #keras.callbacks.ModelCheckpoint('microfaune-' + date_str +'-{epoch:02d}.h5',
        #                          save_weights_only=False)
    ]
    
    #validation_data=(X_test, Y_test),
    
    history = model_extended.fit(data_generator, epochs=4,batch_size=1,workers=6,
                                  #validation_data=(X_test, Y_test), class_weight={0: alpha, 1: 1-alpha},callbacks=micro_callbacks,
                                   verbose=2)
    
    #model.save(f"model-{date_str}")
    #model.save_weights(f"model_weights-{date_str}.h5")

In [ ]:

data_generator = DataGenerator(X_train, Y_train, batch_size, output_size=len(species_of_interests))

sub_data = data_generator.__getitem__(20585)
print("=================")
print(sub_data[0])

# Testing Model

In [ ]:
from datetime import datetime
class ValidDataset(keras.utils.Sequence):
    def __init__(self, X, Y, batch_size=32, output_size=10):
        self.X = X
        self.Y = Y
        #self.UIDs = UIDs
        self.n = len(Y)
        self.batch_size = batch_size
        self.output_size = output_size
        self.shuffle()
        
        
    def __len__(self):
        print("RUNNINGS", int(np.floor(self.n)/self.batch_size))
        return int(np.floor(self.n)/self.batch_size)
    
    def len_of_labels(self):
        return self.output_size
    
    def __getitem__(self, index):
        
        batch_inds = self.inds[self.batch_size*index:self.batch_size*(index+1)]
        tensor = tf.convert_to_tensor(self.X[batch_inds, ...][0])
        #print(index, tf.reduce_max(tensor), tf.reduce_min(tensor), tf.reduce_max(tensor) == tf.reduce_min(tensor))
        if (tf.reduce_max(tensor) == tf.reduce_min(tensor)):
            try:
                print("REDO")
                print(index, tf.reduce_max(tensor), tf.reduce_min(tensor), tf.reduce_max(tensor) == tf.reduce_min(tensor))
                batch_inds = self.inds[self.batch_size*(index+1):self.batch_size*(index+ 2)]
                tensor = tf.convert_to_tensor(self.X[batch_inds, ...][0])
                print(index+1, tf.reduce_max(tensor), tf.reduce_min(tensor), tf.reduce_max(tensor) == tf.reduce_min(tensor))
            except:
                batch_inds = self.inds[self.batch_size*(index-1):self.batch_size*(index)]
                tensor = tf.convert_to_tensor(self.X[batch_inds, ...][0])

        tensor = tf.math.divide(
           tf.math.subtract(
              tensor, 
              tf.reduce_min(tensor)
           ), 
           tf.math.subtract(
              tf.reduce_max(tensor), 
              tf.reduce_min(tensor)
           )
        )
        
        
        
        #print(batch_inds)
        self.counter += self.batch_size
        if self.counter >= self.n:
            self.shuffle()
            
        #TODO: FIX SO BATCH IS MORE THAN 1
        raw_labels = np.array([self.Y[batch_inds][0]])
        formatted_labels = np.zeros(self.output_size)
        formatted_labels[self.Y[batch_inds][0]] = 1
        fprmatted_labels = np.array([formatted_labels])
        #print(formatted_labels.shape)
        #print(np.array([formatted_labels]).shape)
        #print(self.X[batch_inds, ...][0].shape)

        
        #print(tf.convert_to_tensor(self.X[batch_inds, ...][0]), tf.convert_to_tensor(np.array([formatted_labels])))
        return tensor, tf.convert_to_tensor(np.array([formatted_labels]))
    
    
    def shuffle(self):
        self.inds = np.random.permutation(self.n)
        self.counter = 0
# if train:

In [ ]:
validate_dataset = ValidDataset(X_test, Y_test, 1, output_size=11)

In [ ]:
 label_df = pd.DataFrame(columns=range(validate_dataset.len_of_labels()))

In [ ]:
#Label_df_one_hot_encoding
label_df = pd.DataFrame(columns=range(validate_dataset.len_of_labels()))
scores_df = pd.DataFrame(columns=range(validate_dataset.len_of_labels()))
preds_df = pd.DataFrame(columns=range(validate_dataset.len_of_labels()))
for i in range(len(validate_dataset)):
    #print(i)
    predictions = model_extended.predict(
        validate_dataset.__getitem__(i)[0],
        batch_size=None,
        verbose='1',
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=6,
        use_multiprocessing=True
    )
    
    label = predictions.argmax()
    label_arr = np.zeros(validate_dataset.len_of_labels())
    label_arr[label] = 1
    
    preds_df = preds_df.append(pd.DataFrame(np.array([label_arr])))
    scores_df = scores_df.append(pd.DataFrame(predictions))
    label_df = label_df.append(pd.DataFrame(validate_dataset.__getitem__(i)[1].numpy()))
    
preds_df.columns = np.append(species_of_interests, -1)
scores_df.columns = np.append(species_of_interests, -1)
label_df.columns = np.append(species_of_interests, -1)

In [ ]:
preds_df

In [ ]:
scores_df

In [ ]:
label_df

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

In [ ]:
print(classification_report(label_df, preds_df))

In [ ]:
import matplotlib.pyplot as plt
for species in species_of_interests:
    try:
        fpr, tpr, thresh = roc_curve(label_df[species],  scores_df[species])
        auc = roc_auc_score(label_df[species],  preds_df[species])
        plt.plot(fpr,tpr,label="AUC " + species + " "+str(auc))
    except:
        continue

plt.title('Classwise ROC Curves')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
plt.show()

In [ ]:
sum(preds_df.apply(lambda x: sum(x), axis=1))

In [ ]:
preds_df